In [1]:
!sudo apt install tesseract-ocr
!pip install pytesseract

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 39 not upgraded.
Need to get 4,795 kB of archives.
After this operation, 15.8 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr-eng all 4.00~git24-0e00fe6-1.2 [1,588 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr-osd all 4.00~git24-0e00fe6-1.2 [2,989 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr amd64 4.00~git2288-10f4998a-2 [218 kB]
Fetched 4,795 kB in 0s (11.7 MB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5

In [4]:
data_path = '/content/drive/MyDrive/Colab Notebooks/PaySlips_Extraction/Payslips'

In [3]:
import pytesseract

In [4]:
file_path = data_path + '/financial_document_payslip_2019_12_09-15_53_42_611697_.jpg'
print(pytesseract.image_to_string(file_path))

Cognizant Technology Solutions India Private Ltd

Co ni 270 nt Payslip for the month of Mar 2019
g Financial Period 2018-2019
Private & Confidential

Associate Information

Mr. Vaibhav Sawatkar
Associate Id 560560 Manyata (MBP) Bld F3&G4 SEZ

Designation Sr. Analyst - Biz Development PAN DWxxxxxx4N
Male Bank A/C XXXXXXXX4883

saycane Fst Nan
PF A/C TN/MAS/31309/360545 Salary Credited

AN 100743724293 Available Calendar Days 31
SA Policy No - Paid Days 31

 

 

 

Basic 35,250] | ProfessionTax 200

Special Allowance 38,660
Annual Incentive 100,000

(A) Total Earnings 193,450||(B) Total Deduction

 

 

 

Net Salary = (A) - (B) 78,795

Attested

 

Registered Office:Cognizant Technology Solutions India Private Limited
# 5/535, old Mahabalipuram Road, Chennai - 600 097.

Page 1 of 1



In [2]:
!pip install pdfPlumber

In [5]:
import pdfplumber
file_path = data_path + '/financial_document_payslip_2019_12_09-15_53_42_610460_.pdf'
with pdfplumber.open(file_path) as pdf:
    first_page = pdf.pages[0]
    text_from_pdf = first_page.extract_text()

print(text_from_pdf)

text = text_from_pdf

 
CIPLA LIMITED  
CIPLA HOUSE LOWER PAREL MUMBAI - 400013  
 
Pay Slip for the month of June 2019  
 
All amounts in INR  
 
 Emp Code : 107177  Location : Indore
 Emp Name : Sunil Rameshwar Bunkar  IFSC Code : ICIC
 Department : Tablet Manufacturing  Bank A/c No. : 153201512196 (ICICI Bank)
 Designation : Assistant Manager  PAN : AYHPB5362K
 Grade : C I  PF No. : MH/1073/107177
 DOB : 04 Nov 1986    DOJ:03 Nov 2014  Payable Days:30.00   LWP:0.00  PF UAN. : 100474150677
 ESI No. :
           Arrear Day(s) 0.00 Pension Number : MH/1073/91322
                       
Earnings Deductions
Description Rate Monthly Arrear Total Description Amount
 Basic 11951.00  11951.00  0.00  11951.00  PF 1434.00 
 House Rent Allowance 5976.00  5976.00  0.00  5976.00  PROF. TAX 208.00 
 Basket of Allowances 30063.00  30063.00  0.00  30063.00  Canteen_Recovery 150.80 
          Lwf_Ded 10.00 
          Transportation_Recovery 400.00 
 GROSS EARNINGS 47990.00  47990.00  0.00  47990.00   GROSS DEDUCTIONS 2202

In [7]:
! pip install -U spacy -q

     |████████████████████████████████| 6.0 MB 5.1 MB/s 
     |████████████████████████████████| 42 kB 921 kB/s 
     |████████████████████████████████| 653 kB 42.6 MB/s 
     |████████████████████████████████| 10.1 MB 39.6 MB/s 
     |████████████████████████████████| 451 kB 53.0 MB/s 
     |████████████████████████████████| 181 kB 51.1 MB/s 


In [8]:
!python -m spacy info


============================== Info about spaCy ==============================

spaCy version    3.2.4                         
Location         /usr/local/lib/python3.7/dist-packages/spacy
Platform         Linux-5.4.144+-x86_64-with-Ubuntu-18.04-bionic
Python version   3.7.13                        
Pipelines                                      



In [9]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm

nlp = spacy.blank("en") # load a new spacy model
db = DocBin() # create a DocBin object

In [10]:
import json
f = open('/content/drive/MyDrive/Colab Notebooks/PaySlips_Extraction/train_text.json')
TRAIN_DATA = json.load(f)

In [11]:
import json
f = open('/content/drive/MyDrive/Colab Notebooks/PaySlips_Extraction/test_text.json')
TEST_DATA = json.load(f)

In [12]:
for text, annot in tqdm(TRAIN_DATA['annotations']): 
    doc = nlp.make_doc(text) 
    ents = []
    for start, end, label in annot["entities"]:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    doc.ents = ents 
    db.add(doc)

db.to_disk("./training_data.spacy") # save the docbin object

100%|██████████| 220/220 [00:00<00:00, 1711.35it/s]

Skipping entity
Skipping entity


In [13]:
for text, annot in tqdm(TEST_DATA['annotations']): 
    doc = nlp.make_doc(text) 
    ents = []
    for start, end, label in annot["entities"]:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    doc.ents = ents 
    db.add(doc)

db.to_disk("./test_data.spacy") # save the docbin object

100%|██████████| 17/17 [00:00<00:00, 1360.44it/s]


In [14]:
! python -m spacy init config config.cfg --lang en --pipeline ner --optimize efficiency -F

⚠ To generate a more effective transformer-based config (GPU-only),
install the spacy-transformers package and re-run this command. The config
generated now does not use transformers.
ℹ Generated config template specific for your use case
- Language: en
- Pipeline: ner
- Optimize for: efficiency
- Hardware: CPU
- Transformer: None
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [17]:
! python -m spacy train config.cfg --output ./ --paths.train ./training_data.spacy --paths.dev ./test_data.spacy

ℹ Saving to output directory: .
ℹ Using CPU

=========================== Initializing pipeline ===========================
[2022-03-31 14:52:03,592] [INFO] Set up nlp object from config
[2022-03-31 14:52:03,605] [INFO] Pipeline: ['tok2vec', 'ner']
[2022-03-31 14:52:03,610] [INFO] Created vocabulary
[2022-03-31 14:52:03,612] [INFO] Finished initializing nlp object
[2022-03-31 14:52:03,979] [INFO] Initialized pipeline components: ['tok2vec', 'ner']
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     66.56    2.86    3.41    2.46    0.03
 12     200        629.36   5924.16   92.01   92.83   91.20    0.92
 27     400        222.15    462.00   97.35   97.86   96.83    0.97
 46     600        160.74    208.34   97.16   97

In [18]:
nlp_ner = spacy.load("/content/model-best")

In [23]:
doc = nlp_ner(text)

In [24]:
spacy.displacy.render(doc, style="ent", jupyter=True) # display in Jupyter

In [28]:
import pickle

In [29]:
# save the model to disk
filename = '/content/drive/MyDrive/Colab Notebooks/PaySlips_Extraction/finalized_model.sav'
pickle.dump(nlp_ner, open(filename, 'wb'))

In [6]:
! pip install -U spacy -q

import pickle
import spacy

# load the model from disk
filename = '/content/drive/MyDrive/Colab Notebooks/PaySlips_Extraction/finalized_model.sav'
loaded_model = pickle.load(open(filename, 'rb'))

# Display results from the model
result = loaded_model(text)
spacy.displacy.render(result, style="ent", jupyter=True) # display in Jupyter

     |████████████████████████████████| 6.0 MB 5.2 MB/s 
     |████████████████████████████████| 451 kB 67.0 MB/s 
     |████████████████████████████████| 10.1 MB 41.3 MB/s 
     |████████████████████████████████| 42 kB 1.1 MB/s 
     |████████████████████████████████| 181 kB 57.9 MB/s 
     |████████████████████████████████| 653 kB 58.3 MB/s 
